You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files...

You'll work with the following data in the `datasets` folder:
- __Office addresses__
    - Saved in `office_addresses.csv`. 
    - If the value for office is `NaN`, then the employee is remote.
- __Employee addresses__
    - Saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__ 
    - Saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`. 
    - However, this sheet was edited at some point, and ***the headers were removed***! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__ 
    - This information has been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file:
```
{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}
```

In [76]:
import pandas as pd
# Start coding here... 

office = pd.read_csv("datasets/office_addresses.csv")
office



,office,office_country,office_city,office_street,office_street_number
0,Leuven Office,BE,Leuven,Martelarenlaan,38
1,ESB Office,US,New York City,Fifth Avenue,350
2,WeWork Office,GB,London,Old Street,207


In [77]:
employee = pd.read_excel("datasets/employee_information.xlsx")
#print(employee.columns)

employee = employee[['employee_id', 'employee_first_name', 'employee_last_name', 
       'employee_country', 'employee_city', 'employee_street',
       'employee_street_number']]

employee.head()

,employee_id,employee_first_name,employee_last_name,employee_country,employee_city,employee_street,employee_street_number
0,A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9
1,H8K0L6,Tara,Siff,GB,London,Baker Street,221
2,G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66
3,M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7


In [78]:
# Handling contacts datasets

contacts = pd.read_excel("datasets/employee_information.xlsx", sheet_name="emergency_contacts", header=None)
#print(contacts.head())

contacts.rename(columns = {
    0: "employee_id",
    1: "last_name",
    2: "first_name",
    3: "emergency_contact",
    4: "emergency_contact_number",
    5: "relationship"
}, inplace=True)

contacts.head()

,employee_id,last_name,first_name,emergency_contact,emergency_contact_number,relationship
0,A2R5H9,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


In [79]:
# Handling json
roles = pd.read_json("datasets/employee_roles.json")
print(roles.head())

# convert wide format to long format

roles_long = roles.reset_index().melt(id_vars="index", var_name="employee_id", value_name="Value")
roles_long = roles_long.pivot(index="employee_id", columns="index", values="Value").reset_index()
roles_long

                    A2R5H9      H8K0L6              G4R7V0             M1Z7U9
title                  CEO         CFO  Business Developer     Office Manager
monthly_salary       $4500       $4500               $3000              $2000
team            Leadership  Leadership               Sales  People Operations


index,employee_id,monthly_salary,team,title
0,A2R5H9,$4500,Leadership,CEO
1,G4R7V0,$3000,Sales,Business Developer
2,H8K0L6,$4500,Leadership,CFO
3,M1Z7U9,$2000,People Operations,Office Manager


In [80]:
# Merge employee with office
employees = employee.merge(office, left_on="employee_country", right_on="office_country", how="left")
employees

,employee_id,employee_first_name,employee_last_name,employee_country,employee_city,employee_street,employee_street_number,office,office_country,office_city,office_street,office_street_number
0,A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Leuven Office,BE,Leuven,Martelarenlaan,38.0
1,H8K0L6,Tara,Siff,GB,London,Baker Street,221,WeWork Office,GB,London,Old Street,207.0
2,G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66,ESB Office,US,New York City,Fifth Avenue,350.0
3,M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,NaN,NaN,NaN,NaN,NaN


In [81]:
# Merge employees with roles

employees = employees.merge(roles_long, on="employee_id")
employees

,employee_id,employee_first_name,employee_last_name,employee_country,employee_city,employee_street,employee_street_number,office,office_country,office_city,office_street,office_street_number,monthly_salary,team,title
0,A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Leuven Office,BE,Leuven,Martelarenlaan,38.0,$4500,Leadership,CEO
1,H8K0L6,Tara,Siff,GB,London,Baker Street,221,WeWork Office,GB,London,Old Street,207.0,$4500,Leadership,CFO
2,G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66,ESB Office,US,New York City,Fifth Avenue,350.0,$3000,Sales,Business Developer
3,M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,NaN,NaN,NaN,NaN,NaN,$2000,People Operations,Office Manager


In [82]:
# Merge employees with contacts

employees = employees.merge(contacts, on="employee_id")
employees

,employee_id,employee_first_name,employee_last_name,employee_country,employee_city,employee_street,employee_street_number,office,office_country,office_city,office_street,office_street_number,monthly_salary,team,title,last_name,first_name,emergency_contact,emergency_contact_number,relationship
0,A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Leuven Office,BE,Leuven,Martelarenlaan,38.0,$4500,Leadership,CEO,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,Tara,Siff,GB,London,Baker Street,221,WeWork Office,GB,London,Old Street,207.0,$4500,Leadership,CFO,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66,ESB Office,US,New York City,Fifth Avenue,350.0,$3000,Sales,Business Developer,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,NaN,NaN,NaN,NaN,NaN,$2000,People Operations,Office Manager,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


In [83]:
# Handling null values begins with "office"

filtered_cols = [col for col in employees.columns if col.startswith("office")]
filtered_cols

for col in filtered_cols:
    employees[col].fillna("Remote", inplace=True)
    
employees.isna().sum()

employee_id                 0
employee_first_name         0
employee_last_name          0
employee_country            0
employee_city               0
employee_street             0
employee_street_number      0
office                      0
office_country              0
office_city                 0
office_street               0
office_street_number        0
monthly_salary              0
team                        0
title                       0
last_name                   0
first_name                  0
emergency_contact           0
emergency_contact_number    0
relationship                0
dtype: int64

In [84]:
employees_final = employees.set_index("employee_id")
employees_final.head()

,employee_first_name,employee_last_name,employee_country,employee_city,employee_street,employee_street_number,office,office_country,office_city,office_street,office_street_number,monthly_salary,team,title,last_name,first_name,emergency_contact,emergency_contact_number,relationship
employee_id,,,,,,,,,,,,,,,,,,,
A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Leuven Office,BE,Leuven,Martelarenlaan,38.0,$4500,Leadership,CEO,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
H8K0L6,Tara,Siff,GB,London,Baker Street,221,WeWork Office,GB,London,Old Street,207.0,$4500,Leadership,CFO,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66,ESB Office,US,New York City,Fifth Avenue,350.0,$3000,Sales,Business Developer,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,Remote,Remote,Remote,Remote,Remote,$2000,People Operations,Office Manager,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


In [85]:
employees_final.columns

Index(['employee_first_name', 'employee_last_name', 'employee_country',
       'employee_city', 'employee_street', 'employee_street_number', 'office',
       'office_country', 'office_city', 'office_street',
       'office_street_number', 'monthly_salary', 'team', 'title', 'last_name',
       'first_name', 'emergency_contact', 'emergency_contact_number',
       'relationship'],
      dtype='object')

In [86]:
# Rearange columns position

employees_final = employees_final[['employee_first_name', 'employee_last_name', 'employee_country',
       'employee_city', 'employee_street', 'employee_street_number', 'emergency_contact', 'emergency_contact_number', 'relationship', 'monthly_salary', 'team', 'title', 'office','office_country', 'office_city', 'office_street', 'office_street_number']]

employees_final
       
       

,employee_first_name,employee_last_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,monthly_salary,team,title,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,
A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,$4500,Leadership,CEO,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Tara,Siff,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,$4500,Leadership,CFO,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,$3000,Sales,Business Developer,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,$2000,People Operations,Office Manager,Remote,Remote,Remote,Remote,Remote
